#### ID 2120

```The marketing team is evaluating the performance of their previously ran promotions.They are especially interested in comparing the sales on the first day vs. the last day of each promotion. Segment the results by promotion and find the percentage of transactions that happened on the first and last day of each. Your output should contain the promotion ID, percentage of transactions on the first day, and percentage of transactions on the last day.```

In [ ]:
%%sql
SELECT oo.promotion_id,
       100.0 * AVG(CASE WHEN date = start_date THEN 1 ELSE 0 END) AS pct_first_day,
       100.0 * AVG(CASE WHEN date = end_date THEN 1 ELSE 0 END)   AS pct_first_day
FROM online_sales_promotions AS osp
         JOIN online_orders AS oo ON osp.promotion_id = oo.promotion_id
GROUP BY oo.promotion_id

In [ ]:
df = pd.merge(online_sales_promotions, online_orders, how='inner', on='promotion_id')

# df['is_start'] = df.apply(lambda row: 1 if row['start_date'] == row['date'] else 0, axis=1)
# df['is_end'] = df.apply(lambda row: 1 if row['end_date'] == row['date'] else 0, axis=1)

df['is_start'] = (df['start_date'] == df['date']).astype('int')
df['is_end'] = (df['end_date'] == df['date']).astype('int')

result = df.groupby('promotion_id', as_index=False).agg(
    is_start_date=pd.NamedAgg(column='is_start', aggfunc=lambda series: series.mean() * 100),
    is_end_date=pd.NamedAgg(column='is_end', aggfunc=lambda series: series.mean() * 100)
)

#### ID 2121

```The marketing department is assessing the success of their promotional campaigns. You have been asked to find which products sold the most units for each promotion. Your output should contain the promotion ID, product ID, and corresponding total sales for the most successful product ID. In the case of a tie, output all results.```

In [ ]:
%%sql
WITH cte AS (SELECT promotion_id,
                    product_id,
                    SUM(units_sold)                                                AS total_sales,
                    RANK()
                    OVER (PARTITION BY promotion_id ORDER BY SUM(units_sold) DESC) AS rnk
             FROM online_orders
             GROUP BY promotion_id, product_id)
SELECT promotion_id, product_id, total_sales
FROM cte
WHERE rnk = 1

In [ ]:
df = online_orders

grouped_df = df.groupby(['promotion_id', 'product_id'], as_index=False).agg(total_units=('units_sold', 'sum'))

grouped_df['rnk'] = grouped_df.groupby('promotion_id')['total_units'].rank(method='dense', ascending=False)

grouped_df.query('rnk == 1').drop(columns='rnk')

#### ID 2127

```Calculate the sales revenue for the year 2021.```

In [ ]:
%%sql
SELECT SUM(order_total) AS revenue
FROM amazon_sales
WHERE DATE_TRUNC('year', order_date) = '2021-01-01'

In [ ]:
df = amazon_sales

df['year'] = df['order_date'].dt.year

df.query('year == 2021')['order_total'].sum()

#### ID 2129

```You are given a list of posts of a Facebook user. Find the average number of likes.```

In [ ]:
%%sql
SELECT AVG(no_of_likes) AS avg_likes
FROM fb_posts

In [ ]:
df = fb_posts

df['no_of_likes'].mean()